# Termíny odevzdávání projektů
V tomto úkolu si projdeme a vizualizujeme to, v jakém pořadí jste odevzdávali první projekt a první dva bonusové úkoly.

In [ ]:
###
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

Načteme předzpracovaná data z informačního systému. Byla vytvořená jako copy-paste tabulky v HTML do Excelu a tento excelovský soubor byl načten a data byla spojena podle indexů.

Z důvodu ochrany vašeho soukromí byl váš login zahashován (se solí). Dataset ukazuje **datum posledního změny souborů**.

In [ ]:
###
# Načteme dataset
df = pd.read_pickle("odevzdani.pkl.gz")
df

Pandas sám správně interpretoval Excelovské tabuky a také si můžete všimnout *NaT* dat - Not a Time - obdoba NaN.

## Úkol 1: kdy se odevzdávaly projekty?

In [ ]:
# Vytvoříme si kopii datasetu, seřadíme jej podle odevzdání prvního projektu
# vytvoříme pomocný sloupec a integrujeme
df2 = df.copy() ###
df2["odevzdano"] = 1
df2 = df2.sort_values("date_proj1")
df2["odevzdano"] = df2["odevzdano"].cumsum()
df2 ###

In [ ]:
# Zobrazíme data pomocí seabornu
plt.figure(figsize=(12, 4)) ###
ax = sns.lineplot(data=df2, x="date_proj1", y="odevzdano")

# trik pro změnu formátování 
xticks = ax.get_xticks() ###
print("xticks", xticks) ###
ax.set_xticklabels([pd.to_datetime(tm, unit='d').strftime('%d. %m.') for tm in xticks]) ###
plt.show() ###

In [ ]:
# a ted chceme vsechny tri projekty
# muzeme pracovat na urovni dnu - kolik bylo ktery den odevzdano projektu?
display(df.resample("D", on="date_bonus1").agg({"login": "count"}))
display(df.resample("D", on="date_bonus2").agg({"login": "count"}))
display(df.resample("D", on="date_proj1").agg({"login": "count"}))

In [ ]:
# Je mozne to udelat nějak dohromady?
# převedeme tabulku na stacked format
# vytvoříme sloupec cnt
# uděláme groupby 
# a s nějakým krokem  resamplujeme a zjisíme počet
# na konec resetujeme index
df3 = df.melt([],  ["date_bonus1", "date_bonus2", "date_proj1"]).eval("cnt=1").groupby("variable").resample("1D", on="value").count()["cnt"].reset_index()
df3 ###



In [ ]:
# a ted chceme vsechny tri projekty
g = sns.catplot(data = df3, x="value", y="cnt", row="variable", kind="bar", aspect=4)

for ax in g.axes.flat: ###
    xticks = ax.get_xticks() ###
    ax.set_xticklabels([pd.to_datetime(tm, unit='d').strftime('%d. %m.') for tm in xticks]) ###


## Úkol 2: kolik bylo odevzdáno projektů každý den?

In [ ]:
# do df3 vytvoříme sloupec total, který bude odpovídat celkovému počtu
df3["total"] = df3.groupby("variable")["cnt"].cumsum()
df3

In [ ]:
# vykreslíme graf
# pomocí drawstyle="steps-post" z lineárního grafu uděláme schodovitý
sns.relplot(data=df3, kind="line", x="value", y="total", hue="variable", drawstyle="steps-post") # todo: zmenit cetnost na 4 hodiny

## Úkol 3: odevzdávaly se úkoly na poslední chvíli?

In [ ]:
###
# vytvořme si dataset deadlinů jednotlivých projektů
df_terminy = pd.DataFrame(
    {
        "proj": ["date_bonus1", "date_bonus2", "date_proj1"],
        "deadline": [
            np.datetime64("2020-09-25T00:00"),
            np.datetime64("2020-10-19T00:00"),
            np.datetime64("2020-11-12T00:00")
        ]
    }
)
df_terminy



In [ ]:
# Vytvořme df4, který:
# bude zase stacked
# a přidáme mu sloupec deadline (pomocí merge)
df4 = df.melt(["login"], ["date_bonus1", "date_bonus2", "date_proj1"]).merge(df_terminy, left_on="variable", right_on="proj")

#  vytvoříme sloupec reserve, který bude říkat, jaká byla časová rezerv
# a také sloupec reserve_days, který přepočítá tuto hodnotu na dny (float číslo)
df4["reserve"] = df4["deadline"] - df4["value"]
df4["reserve_days"] = df4["reserve"] / np.timedelta64(1, 'D')

df4 

In [ ]:
###
# a koreluje termín odevzdání?
df5 = df4.pivot("login", "proj", "reserve_days")
df5



In [ ]:
###
# a znázorníme korelaci
def diag_line(x, y, **kwds):
    plt.plot([x.min(), x.max()], [y.min(), y.max()], color="tab:red", lw=1, ls=":")
    sns.scatterplot(x, y, **kwds)


g = sns.PairGrid(data=df5)
g.map_lower(diag_line)
g.map_upper(diag_line)
g.map_diag(sns.histplot, kde=True, bins=20)
#g.map_lower(sns.scatterplot)
g.fig.show()


Co v datech vidíme?
* člověk, který odevzdal projekt dost dopředu tak odevzdal dost dopředu i bonus2
* vzhledem k téměř prázdné spodní polovině u date_bonus2 vs date_proj1 při projektu měli lidé méně času
* byla by zajímavá korelace s počtem bodů, to však nebudeme ukazovat :)